## Статистика всех поисковых фраз за период по дням

Скрипт выгружает визиты и посетителей в разбивке по поисковым фразам и системам для заданного счетчика Яндекс Метрики. Выгрузка происходит за каждый день периода по-отдельности. Итоговая выгрузка пишется в файл data.txt

In [1]:
import requests
from datetime import datetime, timedelta

### Параметры работы скрипта

In [2]:
# начальная и конечная дата выгрузки

startDate = '2017-08-01'
endDate = '2017-08-10'

In [3]:
counterID = '' # номер счетчика

In [10]:
# измерения и метрики, из которых формируем отчет

dimensions = [ 'ym:s:<attribution>SearchEngineRoot', 'ym:s:<attribution>SearchPhrase' ]
metrics = [ 'ym:s:visits', 'ym:s:users' ]

# пример добавления третьего измерения в выгрузку
# dimensions = [ 'ym:s:<attribution>SearchEngineRoot', 'ym:s:<attribution>SearchPhrase', 'ym:s:regionCountry' ]

In [5]:
# токен для обращения к API
# как получить токен описано в документации https://tech.yandex.ru/metrika/doc/api2/intro/authorization-docpage/
# для простоты используйте опцию получения отладочного токена https://tech.yandex.ru/oauth/doc/dg/tasks/get-oauth-token-docpage/

token = ''

In [6]:
# URL для получения отчетов, в текущей версии Яндекс Метрики не меняется
metrikaAPIurl = 'https://api-metrika.yandex.ru/stat/v1/data'

In [7]:
def makeRequest( counterID, currentDT, offset, limit ):
    """
    Функция делает запрос к API и возвращает содержимое элемента 'data' ответа API.
    В случае ошибки выдает 'error'
    """
    
    requestParams = {
        'dimensions': ','.join( dimensions ),
        'metrics': ','.join( metrics ),
        'id': counterID,
        'date1': currentDT.strftime('%Y-%m-%d'),
        'date2': currentDT.strftime('%Y-%m-%d'),
        'accuracy': '1',
        'limit': limit,
        'offset': offset,
        'oauth_token': token
    }
    
    r = requests.get( metrikaAPIurl, params = requestParams )
    
    if 'errors' in r.json():
        print( 'Что-то пошло не так для счетчика {} за дату {}. Вот что известно: {}'.format( counterID, currentDT.strftime('%Y-%m-%d'), r.json()['errors'] ) )
        return 'error'
    else:
        return r.json()['data']

In [11]:
# переводим начальную и конечную дату выгрузки из типа string в datetime
# с помощью типа datetime можно будет проходить в цикле между этими датами, прибавляя 1 день
startDateDT = datetime.strptime( startDate, '%Y-%m-%d' )
endDateDT = datetime.strptime( endDate, '%Y-%m-%d' )

# текущий день при проходе между начальной и конечной датой в цикле
# начинаем, ествественно, с начальной датой startDate
currentDT = startDateDT

# количество строк, которые выгружаем за один запрос к API
# максимальное значение 100000, лучше ставить сразу его, чтобы лишний раз не обращаться к API
# меньшие значения подойдут для проверки корректности выгрузки
limit = 100000

# открываем файл data.txt для записи новых значений и удаляем его содержимое (опция 'w')
with open( 'data.txt', 'w' ) as f:
    
    # переменная currentDT будет последовательно проходить значения от startDate до endDate
    while currentDT <= endDateDT:
        print( 'Загрузка данных за ' + currentDT.strftime('%Y-%m-%d') )

        # заранее неизвестно сколько строк в отчете по поисковым фразам за текущую дату currentDT
        # поэтому нам нужна переменная, которая при получении пустого отчета (т. е. мы выгрузили всю таблицу) 
        # будет "сообщать" скрипту, что выгрузка завершена и пора переходить к следующей дате
        Flag = False
        
        # номер строки отчета, который собираемся выгружать
        # в каждом запросе к API за текущую дату currentDT это значение будет увеличиваться на число выгруженных строк limit
        offset = 1
        
        # выгружаем limit-строк из отчета за текущую дату currentDT, начиная с позиции offset
        # затем увеличиваем значение offset на значение limit и повторяем запрос
        while not Flag:
            
            # вызываем функцию выгрузки отчета за дату currentDT, начиная со строки offset
            data = makeRequest( counterID, currentDT, offset, limit )

            # проверяем, что в запросе не было ошибки и мы получили какие-то строки
            # если функция вернет на 0 строк, значит мы выгрузили всю таблицу за текущую дату
            if data != 'error' and len( data ) > 0:
                
                # в случае корректного ответа каждая строчка row имеет вид
                # {'dimensions': [{'name': 'Google', 'id': 'google', 'favicon': 'google.com'}, {'name': None, 'url': None, 'favicon': None}], 'metrics': [191.0, 175.0]}
                # для каждой строчки в таблице извлекаем ее столбцы и пишем результат в файл
                for row in data:
                    
                    # список названий измерений
                    # в примере это [ 'Google', 'None' ]
                    dimensionsNames = [ str( x['name'] ) for x in row['dimensions'] ]
                    
                    # значения метрик
                    # в примере это [ '191.0', '175.0' ]
                    metricsValues = [ '{:.0f}'.format( x ) for x in row['metrics'] ]

                    # если надо вывести результат на экран раскомментируйте строчку ниже
                    # print( '\t'.join( dimensionsNames ) + '\t' + '\t'.join( metricsValues ) )
                    
                    # пишем результат в файл
                    f.write( '{}\t{}\t{}\n'.format( currentDT.strftime('%Y-%m-%d'), '\t'.join( dimensionsNames ), '\t'.join( metricsValues ) ) )

                offset += limit

            else:
                Flag = True


        currentDT += timedelta( days = 1 )

print( 'Результаты выгружены в файл data.txt' )

Загрузка данных за 2017-08-01
Загрузка данных за 2017-08-02
Загрузка данных за 2017-08-03
Загрузка данных за 2017-08-04
Загрузка данных за 2017-08-05
Загрузка данных за 2017-08-06
Загрузка данных за 2017-08-07
Загрузка данных за 2017-08-08
Загрузка данных за 2017-08-09
Загрузка данных за 2017-08-10
Результаты выгружены в файл data.txt
